In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [2]:
rr = RandomRules(
  TOTAL_NUMBERS = 40,
  CHOOSE = 6,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5, 6],
  MAX_B_VALUES = [35, 36, 37, 38, 39, 40]
)


In [3]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=42'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [4]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_b5 = None
post_str_blast = None
for row in rows:
  for col_index in [2, 3]:
    cols = [data.text.strip() for data in row.find_all('td')]
    if (cols[col_index] == ''):
      continue

    # cols = [data.text.strip() for data in row.find_all('td')]
    
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_b2)
    cols.append(post_str_b3)
    cols.append(post_str_b4)
    cols.append(post_str_b5)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_b2 = str_combo[1]
    post_str_b3 = str_combo[2]
    post_str_b4 = str_combo[3]
    post_str_b5 = str_combo[4]
    post_str_blast = str_combo[-1]



# lst = []
# for row in rows:
  
  
#   for col_index in [2, 3]:
#     cols = [data.text.strip() for data in row.find_all('td')]
#     if (cols[col_index] == ''):
#       continue
  
#     id, bolillas_int = formalize_data(bolillas_str=cols[col_index])
#     cols.append(id)
#     cols.append(bolillas_int)
#     cols.extend([b for b in bolillas_int])
    
#     is_rare, reason = rr.is_rare_combination(bolillas_int, id)
#     cols.append(is_rare)
#     cols.append(reason)
#     left_scores = [round(b, 4) for b in rr.calculate_left_scores(bolillas_int)]

#     right_scores = [round(b, 4) for b in rr.calculate_right_scores(bolillas_int)]
#     lr_scores = [round(left + right,4) for left, right in zip(left_scores, right_scores)]
#     row_score = round(sum(lr_scores),4)

#     shared_values = list(set(left_scores[0:5]) & set(right_scores[1:6]))
#     shared_values = [round(s, 4) for s in shared_values]
#     cols.extend([score for score in left_scores])
#     cols.extend([score for score in right_scores])
#     # cols.extend(lr_scores)
#     cols.append(shared_values)
#     cols.append(len(shared_values))
#     cols.append(row_score)
#     cols.append(rr.jumps_map(bolillas_int))
#     cols.append(rr.unijump(bolillas_int))
    
#     lst.append(cols)

with open('./kbl_cache_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=5288


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/04/2025""","""1800""","""20 35 03 19 09 29""","""09 17 29 39 31 34""","""""","""""","""030919202935""","[3, 9, … 35]",3,9,19,20,29,35,"""03""","""09""","""19""","""20""","""29""","""35""",null,null,null,null,null,null,false,0,"""06_10_01_09_06""",-58
"""26/04/2025""","""1800""","""20 35 03 19 09 29""","""09 17 29 39 31 34""","""""","""""","""030919202935""","[3, 9, … 35]",3,9,19,20,29,35,"""03""","""09""","""19""","""20""","""29""","""35""","""03""","""09""","""19""","""20""","""29""","""35""",false,0,"""06_10_01_09_06""",-58
"""24/04/2025""","""1799""","""16 14 17 34 30 40""","""33 17 18 39 03 09""","""""","""""","""141617303440""","[14, 16, … 40]",14,16,17,30,34,40,"""14""","""16""","""17""","""30""","""34""","""40""","""03""","""09""","""19""","""20""","""29""","""35""",true,16,"""02_01_13_04_06""",66
"""24/04/2025""","""1799""","""16 14 17 34 30 40""","""33 17 18 39 03 09""","""""","""""","""141617303440""","[14, 16, … 40]",14,16,17,30,34,40,"""14""","""16""","""17""","""30""","""34""","""40""","""14""","""16""","""17""","""30""","""34""","""40""",true,16,"""02_01_13_04_06""",66
"""22/04/2025""","""1798""","""33 18 12 13 27 24""","""17 40 34 23 18 32""","""""","""""","""121318242733""","[12, 13, … 33]",12,13,18,24,27,33,"""12""","""13""","""18""","""24""","""27""","""33""","""14""","""16""","""17""","""30""","""34""","""40""",false,0,"""01_05_06_03_06""",11
"""22/04/2025""","""1798""","""33 18 12 13 27 24""","""17 40 34 23 18 32""","""""","""""","""121318242733""","[12, 13, … 33]",12,13,18,24,27,33,"""12""","""13""","""18""","""24""","""27""","""33""","""12""","""13""","""18""","""24""","""27""","""33""",false,0,"""01_05_06_03_06""",11
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""12""","""13""","""18""","""24""","""27""","""33""",false,0,"""02_01_18_12_06""",64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""02_01_18_12_06""",64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""04_06_03_22_01""",-89


In [5]:
dfRows.item(0, 'b2')

9

In [6]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [7]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='03'
df_viz FOR str_b1 = 03


post_str_b1,count,percent
str,u32,f64
"""03""",267,40.271493
"""02""",69,10.40724
"""01""",60,9.049774
"""05""",46,6.93816
"""04""",40,6.033183
…,…,…
"""20""",2,0.301659
"""16""",2,0.301659
"""21""",1,0.15083


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 03
Last 3 results last_results=['05', '03']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/04/2025""","""1800""","""20 35 03 19 09 29""","""09 17 29 39 31 34""","""""","""""","""030919202935""","[3, 9, … 35]",3,9,19,20,29,35,"""03""","""09""","""19""","""20""","""29""","""35""","""03""","""09""","""19""","""20""","""29""","""35""",false,0,"""06_10_01_09_06""",-58
"""12/04/2025""","""1794""","""10 32 26 03 19 11""","""39 07 28 21 05 12""","""""","""""","""031011192632""","[3, 10, … 32]",3,10,11,19,26,32,"""03""","""10""","""11""","""19""","""26""","""32""","""05""","""08""","""13""","""21""","""34""","""37""",false,0,"""07_01_08_07_06""",29
"""12/04/2025""","""1794""","""10 32 26 03 19 11""","""39 07 28 21 05 12""","""""","""""","""031011192632""","[3, 10, … 32]",3,10,11,19,26,32,"""03""","""10""","""11""","""19""","""26""","""32""","""03""","""10""","""11""","""19""","""26""","""32""",false,0,"""07_01_08_07_06""",29
"""25/03/2025""","""1786""","""31 03 36 24 14 33""","""21 40 02 16 05 27""","""""","""""","""031424313336""","[3, 14, … 36]",3,14,24,31,33,36,"""03""","""14""","""24""","""31""","""33""","""36""","""06""","""10""","""16""","""23""","""26""","""32""",false,0,"""11_10_07_02_03""",8
"""25/03/2025""","""1786""","""31 03 36 24 14 33""","""21 40 02 16 05 27""","""""","""""","""031424313336""","[3, 14, … 36]",3,14,24,31,33,36,"""03""","""14""","""24""","""31""","""33""","""36""","""03""","""14""","""24""","""31""","""33""","""36""",false,0,"""11_10_07_02_03""",8
"""20/03/2025""","""1784""","""13 07 35 15 03 11""","""28 12 22 35 08 37""","""""","""""","""030711131535""","[3, 7, … 35]",3,7,11,13,15,35,"""03""","""07""","""11""","""13""","""15""","""35""","""07""","""23""","""24""","""25""","""29""","""34""",true,27,"""04_04_02_02_20""",12
"""20/03/2025""","""1784""","""13 07 35 15 03 11""","""28 12 22 35 08 37""","""""","""""","""030711131535""","[3, 7, … 35]",3,7,11,13,15,35,"""03""","""07""","""11""","""13""","""15""","""35""","""03""","""07""","""11""","""13""","""15""","""35""",true,27,"""04_04_02_02_20""",12
"""15/03/2025""","""1782""","""19 28 03 39 10 09""","""06 35 12 28 09 18""","""""","""""","""030910192839""","[3, 9, … 39]",3,9,10,19,28,39,"""03""","""09""","""10""","""19""","""28""","""39""","""12""","""16""","""21""","""22""","""30""","""39""",true,8,"""06_01_09_09_11""",31
"""15/03/2025""","""1782""","""19 28 03 39 10 09""","""06 35 12 28 09 18""","""""","""""","""030910192839""","[3, 9, … 39]",3,9,10,19,28,39,"""03""","""09""","""10""","""19""","""28""","""39""","""03""","""09""","""10""","""19""","""28""","""39""",true,8,"""06_01_09_09_11""",31


df_jumps_map FOR str_b1 = 03


jumps_map,count
str,u32
"""06_01_09_09_11""",2
"""07_07_10_03_08""",2
"""04_09_09_04_04""",2
"""09_05_03_01_19""",2
"""02_04_12_06_05""",2
…,…
"""06_06_09_01_05""",1
"""01_01_11_19_03""",1
"""06_11_07_02_09""",1


df_unijump FOR str_b1 = 03


unijump,count
i64,u32
-2,11
5,10
2,10
27,9
-11,9
…,…
88,1
78,1
-72,1


In [8]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='09'
df_viz FOR str_b2 = 09


post_str_b2,count,percent
str,u32,f64
"""09""",132,35.106383
"""08""",21,5.585106
"""04""",20,5.319149
"""07""",18,4.787234
"""10""",17,4.521277
…,…,…
"""21""",3,0.797872
"""24""",3,0.797872
"""22""",2,0.531915


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 09
Last 3 results last_results=['09', '16']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/04/2025""","""1800""","""20 35 03 19 09 29""","""09 17 29 39 31 34""","""""","""""","""030919202935""","[3, 9, … 35]",3,9,19,20,29,35,"""03""","""09""","""19""","""20""","""29""","""35""","""03""","""09""","""19""","""20""","""29""","""35""",false,0,"""06_10_01_09_06""",-58
"""15/03/2025""","""1782""","""19 28 03 39 10 09""","""06 35 12 28 09 18""","""""","""""","""030910192839""","[3, 9, … 39]",3,9,10,19,28,39,"""03""","""09""","""10""","""19""","""28""","""39""","""12""","""16""","""21""","""22""","""30""","""39""",true,8,"""06_01_09_09_11""",31
"""15/03/2025""","""1782""","""19 28 03 39 10 09""","""06 35 12 28 09 18""","""""","""""","""030910192839""","[3, 9, … 39]",3,9,10,19,28,39,"""03""","""09""","""10""","""19""","""28""","""39""","""03""","""09""","""10""","""19""","""28""","""39""",true,8,"""06_01_09_09_11""",31
"""08/03/2025""","""1779""","""14 07 39 23 30 09""","""33 08 10 36 32 09""","""""","""""","""070914233039""","[7, 9, … 39]",7,9,14,23,30,39,"""07""","""09""","""14""","""23""","""30""","""39""","""06""","""10""","""12""","""15""","""34""","""39""",false,0,"""02_05_09_07_09""",17
"""08/03/2025""","""1779""","""14 07 39 23 30 09""","""33 08 10 36 32 09""","""""","""""","""070914233039""","[7, 9, … 39]",7,9,14,23,30,39,"""07""","""09""","""14""","""23""","""30""","""39""","""07""","""09""","""14""","""23""","""30""","""39""",false,0,"""02_05_09_07_09""",17
"""31/12/2024""","""1750""","""28 25 09 30 12 04""","""23 12 25 39 24 32""","""""","""""","""040912252830""","[4, 9, … 30]",4,9,12,25,28,30,"""04""","""09""","""12""","""25""","""28""","""30""","""01""","""10""","""16""","""24""","""34""","""35""",false,0,"""05_03_13_03_02""",61
"""31/12/2024""","""1750""","""28 25 09 30 12 04""","""23 12 25 39 24 32""","""""","""""","""040912252830""","[4, 9, … 30]",4,9,12,25,28,30,"""04""","""09""","""12""","""25""","""28""","""30""","""04""","""09""","""12""","""25""","""28""","""30""",false,0,"""05_03_13_03_02""",61
"""12/11/2024""","""1729""","""34 33 38 01 10 09""","""07 33 35 31 26 22""","""""","""""","""010910333438""","[1, 9, … 38]",1,9,10,33,34,38,"""01""","""09""","""10""","""33""","""34""","""38""","""06""","""10""","""14""","""19""","""20""","""27""",true,16,"""08_01_23_01_04""",142
"""12/11/2024""","""1729""","""34 33 38 01 10 09""","""07 33 35 31 26 22""","""""","""""","""010910333438""","[1, 9, … 38]",1,9,10,33,34,38,"""01""","""09""","""10""","""33""","""34""","""38""","""01""","""09""","""10""","""33""","""34""","""38""",true,16,"""08_01_23_01_04""",142


df_jumps_map FOR str_b2 = 09


jumps_map,count
str,u32
"""07_01_17_01_08""",2
"""05_02_02_11_15""",2
"""04_01_01_04_02""",2
"""08_01_08_11_07""",2
"""01_04_06_07_04""",2
…,…
"""02_03_02_12_11""",1
"""04_13_03_05_10""",1
"""01_04_19_03_01""",1


df_unijump FOR str_b2 = 09


unijump,count
i64,u32
17,9
-15,8
2,8
-3,7
18,6
…,…
16,1
-99,1
105,1


In [9]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='19'
df_viz FOR str_b3 = 19


post_str_b3,count,percent
str,u32,f64
"""19""",109,37.201365
"""22""",15,5.119454
"""18""",13,4.43686
"""16""",12,4.095563
"""11""",11,3.754266
…,…,…
"""32""",2,0.682594
"""34""",2,0.682594
"""04""",1,0.341297


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 19
Last 3 results last_results=['19', '10']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/04/2025""","""1800""","""20 35 03 19 09 29""","""09 17 29 39 31 34""","""""","""""","""030919202935""","[3, 9, … 35]",3,9,19,20,29,35,"""03""","""09""","""19""","""20""","""29""","""35""","""03""","""09""","""19""","""20""","""29""","""35""",false,0,"""06_10_01_09_06""",-58
"""13/03/2025""","""1781""","""21 19 06 07 28 40""","""16 15 28 27 25 05""","""""","""""","""060719212840""","[6, 7, … 40]",6,7,19,21,28,40,"""06""","""07""","""19""","""21""","""28""","""40""","""03""","""09""","""10""","""19""","""28""","""39""",true,16,"""01_12_02_07_12""",-51
"""13/03/2025""","""1781""","""21 19 06 07 28 40""","""16 15 28 27 25 05""","""""","""""","""060719212840""","[6, 7, … 40]",6,7,19,21,28,40,"""06""","""07""","""19""","""21""","""28""","""40""","""06""","""07""","""19""","""21""","""28""","""40""",true,16,"""01_12_02_07_12""",-51
"""23/01/2025""","""1760""","""34 25 14 27 10 19""","""36 12 35 30 40 09""","""""","""""","""101419252734""","[10, 14, … 34]",10,14,19,25,27,34,"""10""","""14""","""19""","""25""","""27""","""34""","""17""","""23""","""32""","""33""","""34""","""35""",false,0,"""04_05_06_02_07""",19
"""23/01/2025""","""1760""","""34 25 14 27 10 19""","""36 12 35 30 40 09""","""""","""""","""101419252734""","[10, 14, … 34]",10,14,19,25,27,34,"""10""","""14""","""19""","""25""","""27""","""34""","""10""","""14""","""19""","""25""","""27""","""34""",false,0,"""04_05_06_02_07""",19
"""11/01/2025""","""1755""","""05 24 25 30 13 19""","""35 18 28 22 27 05""","""""","""""","""051319242530""","[5, 13, … 30]",5,13,19,24,25,30,"""05""","""13""","""19""","""24""","""25""","""30""","""02""","""16""","""18""","""23""","""31""","""34""",false,0,"""08_06_05_01_05""",15
"""11/01/2025""","""1755""","""05 24 25 30 13 19""","""35 18 28 22 27 05""","""""","""""","""051319242530""","[5, 13, … 30]",5,13,19,24,25,30,"""05""","""13""","""19""","""24""","""25""","""30""","""05""","""13""","""19""","""24""","""25""","""30""",false,0,"""08_06_05_01_05""",15
"""04/01/2025""","""1752""","""27 37 08 39 19 12""","""40 02 30 07 09 03""","""""","""""","""081219273739""","[8, 12, … 39]",8,12,19,27,37,39,"""08""","""12""","""19""","""27""","""37""","""39""","""04""","""06""","""07""","""08""","""27""","""38""",false,0,"""04_07_08_10_02""",-14
"""04/01/2025""","""1752""","""27 37 08 39 19 12""","""40 02 30 07 09 03""","""""","""""","""081219273739""","[8, 12, … 39]",8,12,19,27,37,39,"""08""","""12""","""19""","""27""","""37""","""39""","""08""","""12""","""19""","""27""","""37""","""39""",false,0,"""04_07_08_10_02""",-14


df_jumps_map FOR str_b3 = 19


jumps_map,count
str,u32
"""06_07_09_02_01""",2
"""02_08_01_06_02""",2
"""01_12_02_07_12""",2
"""11_06_08_09_04""",2
"""09_04_18_02_01""",2
…,…
"""02_04_01_17_03""",1
"""05_09_01_07_09""",1
"""09_05_10_06_05""",1


df_unijump FOR str_b3 = 19


unijump,count
i64,u32
14,8
29,7
-31,7
-11,6
-39,6
…,…
23,1
-62,1
65,1


In [10]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='20'
df_viz FOR str_b4 = 20


post_str_b4,count,percent
str,u32,f64
"""20""",95,36.259542
"""29""",18,6.870229
"""23""",15,5.725191
"""19""",12,4.580153
"""27""",11,4.198473
…,…,…
"""16""",3,1.145038
"""11""",3,1.145038
"""08""",2,0.763359


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 20
Last 3 results last_results=['19', '20']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/04/2025""","""1800""","""20 35 03 19 09 29""","""09 17 29 39 31 34""","""""","""""","""030919202935""","[3, 9, … 35]",3,9,19,20,29,35,"""03""","""09""","""19""","""20""","""29""","""35""","""03""","""09""","""19""","""20""","""29""","""35""",false,0,"""06_10_01_09_06""",-58
"""10/04/2025""","""1793""","""01 36 20 18 22 02""","""33 17 29 39 30 36""","""""","""""","""010218202236""","[1, 2, … 36]",1,2,18,20,22,36,"""01""","""02""","""18""","""20""","""22""","""36""","""03""","""10""","""11""","""19""","""26""","""32""",true,8,"""01_16_02_02_14""",-45
"""10/04/2025""","""1793""","""01 36 20 18 22 02""","""33 17 29 39 30 36""","""""","""""","""010218202236""","[1, 2, … 36]",1,2,18,20,22,36,"""01""","""02""","""18""","""20""","""22""","""36""","""01""","""02""","""18""","""20""","""22""","""36""",true,8,"""01_16_02_02_14""",-45
"""11/02/2025""","""1768""","""32 20 15 12 30 11""","""37 26 38 35 02 31""","""""","""""","""111215203032""","[11, 12, … 32]",11,12,15,20,30,32,"""11""","""12""","""15""","""20""","""30""","""32""","""02""","""07""","""08""","""12""","""20""","""27""",false,0,"""01_03_05_10_02""",-19
"""11/02/2025""","""1768""","""32 20 15 12 30 11""","""37 26 38 35 02 31""","""""","""""","""111215203032""","[11, 12, … 32]",11,12,15,20,30,32,"""11""","""12""","""15""","""20""","""30""","""32""","""11""","""12""","""15""","""20""","""30""","""32""",false,0,"""01_03_05_10_02""",-19
"""28/11/2024""","""1736""","""20 26 39 08 19 10""","""09 26 13 23 24 05""","""""","""""","""081019202639""","[8, 10, … 39]",8,10,19,20,26,39,"""08""","""10""","""19""","""20""","""26""","""39""","""03""","""05""","""09""","""10""","""37""","""39""",false,0,"""02_09_01_06_13""",-39
"""28/11/2024""","""1736""","""20 26 39 08 19 10""","""09 26 13 23 24 05""","""""","""""","""081019202639""","[8, 10, … 39]",8,10,19,20,26,39,"""08""","""10""","""19""","""20""","""26""","""39""","""08""","""10""","""19""","""20""","""26""","""39""",false,0,"""02_09_01_06_13""",-39
"""23/11/2024""","""1734""","""29 25 13 20 19 07""","""08 01 32 11 14 12""","""""","""""","""071319202529""","[7, 13, … 29]",7,13,19,20,25,29,"""07""","""13""","""19""","""20""","""25""","""29""","""02""","""06""","""21""","""23""","""32""","""37""",true,9,"""06_06_01_05_04""",-28
"""23/11/2024""","""1734""","""29 25 13 20 19 07""","""08 01 32 11 14 12""","""""","""""","""071319202529""","[7, 13, … 29]",7,13,19,20,25,29,"""07""","""13""","""19""","""20""","""25""","""29""","""07""","""13""","""19""","""20""","""25""","""29""",true,9,"""06_06_01_05_04""",-28


df_jumps_map FOR str_b4 = 20


jumps_map,count
str,u32
"""01_04_01_03_08""",2
"""08_05_03_15_02""",2
"""02_04_12_06_09""",2
"""01_01_04_05_01""",2
"""02_11_04_09_03""",2
…,…
"""03_07_06_02_10""",1
"""14_01_01_13_06""",1
"""10_06_02_01_18""",1


df_unijump FOR str_b4 = 20


unijump,count
i64,u32
10,8
26,7
11,7
-3,7
-20,6
…,…
-67,1
-69,1
68,1


In [11]:
possible_b5, exclude_jump_maps_b5, last_post_b5 = plot_post_col(col='str_b5', post_col='post_str_b5')

Setting test_value='29'
df_viz FOR str_b5 = 29


post_str_b5,count,percent
str,u32,f64
"""29""",114,36.075949
"""33""",16,5.063291
"""32""",15,4.746835
"""31""",14,4.43038
"""35""",14,4.43038
…,…,…
"""17""",1,0.316456
"""16""",1,0.316456
"""14""",1,0.316456


:Overlay
   .Bars.I    :Bars   [post_str_b5]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 29
Last 3 results last_results=['29', '33']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/04/2025""","""1800""","""20 35 03 19 09 29""","""09 17 29 39 31 34""","""""","""""","""030919202935""","[3, 9, … 35]",3,9,19,20,29,35,"""03""","""09""","""19""","""20""","""29""","""35""","""03""","""09""","""19""","""20""","""29""","""35""",false,0,"""06_10_01_09_06""",-58
"""22/03/2025""","""1785""","""25 07 34 29 24 23""","""36 12 20 40 06 17""","""""","""""","""072324252934""","[7, 23, … 34]",7,23,24,25,29,34,"""07""","""23""","""24""","""25""","""29""","""34""","""03""","""14""","""24""","""31""","""33""","""36""",true,24,"""16_01_01_04_05""",7
"""22/03/2025""","""1785""","""25 07 34 29 24 23""","""36 12 20 40 06 17""","""""","""""","""072324252934""","[7, 23, … 34]",7,23,24,25,29,34,"""07""","""23""","""24""","""25""","""29""","""34""","""07""","""23""","""24""","""25""","""29""","""34""",true,24,"""16_01_01_04_05""",7
"""08/02/2025""","""1767""","""34 22 16 17 21 29""","""38 05 33 35 07 06""","""""","""""","""161721222934""","[16, 17, … 34]",16,17,21,22,29,34,"""16""","""17""","""21""","""22""","""29""","""34""","""11""","""12""","""15""","""20""","""30""","""32""",false,0,"""01_04_01_07_05""",-32
"""08/02/2025""","""1767""","""34 22 16 17 21 29""","""38 05 33 35 07 06""","""""","""""","""161721222934""","[16, 17, … 34]",16,17,21,22,29,34,"""16""","""17""","""21""","""22""","""29""","""34""","""16""","""17""","""21""","""22""","""29""","""34""",false,0,"""01_04_01_07_05""",-32
"""28/12/2024""","""1749""","""04 27 33 17 24 29""","""34 25 22 21 04 09""","""""","""""","""041724272933""","[4, 17, … 33]",4,17,24,27,29,33,"""04""","""17""","""24""","""27""","""29""","""33""","""04""","""09""","""12""","""25""","""28""","""30""",false,0,"""13_07_03_02_04""",-1
"""28/12/2024""","""1749""","""04 27 33 17 24 29""","""34 25 22 21 04 09""","""""","""""","""041724272933""","[4, 17, … 33]",4,17,24,27,29,33,"""04""","""17""","""24""","""27""","""29""","""33""","""04""","""17""","""24""","""27""","""29""","""33""",false,0,"""13_07_03_02_04""",-1
"""26/10/2024""","""1722""","""25 11 14 29 36 07""","""18 08 21 06 09 35""","""""","""""","""071114252936""","[7, 11, … 36]",7,11,14,25,29,36,"""07""","""11""","""14""","""25""","""29""","""36""","""01""","""09""","""27""","""31""","""35""","""38""",false,0,"""04_03_11_04_07""",49
"""26/10/2024""","""1722""","""25 11 14 29 36 07""","""18 08 21 06 09 35""","""""","""""","""071114252936""","[7, 11, … 36]",7,11,14,25,29,36,"""07""","""11""","""14""","""25""","""29""","""36""","""07""","""11""","""14""","""25""","""29""","""36""",false,0,"""04_03_11_04_07""",49


df_jumps_map FOR str_b5 = 29


jumps_map,count
str,u32
"""10_04_01_11_08""",2
"""02_11_04_09_03""",2
"""05_05_11_03_09""",2
"""02_10_09_04_06""",2
"""01_06_10_03_03""",2
…,…
"""11_07_01_07_02""",1
"""10_11_03_04_11""",1
"""02_02_04_10_10""",1


df_unijump FOR str_b5 = 29


unijump,count
i64,u32
5,8
-17,7
10,7
6,7
16,7
…,…
13,1
-38,1
-35,1


In [12]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b6', post_col='post_str_blast')

Setting test_value='35'
df_viz FOR str_b6 = 35


post_str_blast,count,percent
str,u32,f64
"""35""",142,38.172043
"""40""",50,13.44086
"""39""",32,8.602151
"""37""",24,6.451613
"""36""",23,6.182796
…,…,…
"""21""",2,0.537634
"""24""",2,0.537634
"""22""",1,0.268817


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b6 = 35
Last 3 results last_results=['34', '35']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/04/2025""","""1800""","""20 35 03 19 09 29""","""09 17 29 39 31 34""","""""","""""","""030919202935""","[3, 9, … 35]",3,9,19,20,29,35,"""03""","""09""","""19""","""20""","""29""","""35""","""03""","""09""","""19""","""20""","""29""","""35""",false,0,"""06_10_01_09_06""",-58
"""20/03/2025""","""1784""","""13 07 35 15 03 11""","""28 12 22 35 08 37""","""""","""""","""030711131535""","[3, 7, … 35]",3,7,11,13,15,35,"""03""","""07""","""11""","""13""","""15""","""35""","""07""","""23""","""24""","""25""","""29""","""34""",true,27,"""04_04_02_02_20""",12
"""20/03/2025""","""1784""","""13 07 35 15 03 11""","""28 12 22 35 08 37""","""""","""""","""030711131535""","[3, 7, … 35]",3,7,11,13,15,35,"""03""","""07""","""11""","""13""","""15""","""35""","""03""","""07""","""11""","""13""","""15""","""35""",true,27,"""04_04_02_02_20""",12
"""25/01/2025""","""1761""","""17 35 23 33 32 34""","""33 02 17 20 03 13""","""""","""""","""172332333435""","[17, 23, … 35]",17,23,32,33,34,35,"""17""","""23""","""32""","""33""","""34""","""35""","""01""","""06""","""14""","""18""","""24""","""39""",true,8,"""06_09_01_01_01""",-27
"""25/01/2025""","""1761""","""17 35 23 33 32 34""","""33 02 17 20 03 13""","""""","""""","""172332333435""","[17, 23, … 35]",17,23,32,33,34,35,"""17""","""23""","""32""","""33""","""34""","""35""","""17""","""23""","""32""","""33""","""34""","""35""",true,8,"""06_09_01_01_01""",-27
"""02/01/2025""","""1751""","""10 34 24 16 35 01""","""11 03 18 29 15 28""","""""","""""","""011016243435""","[1, 10, … 35]",1,10,16,24,34,35,"""01""","""10""","""16""","""24""","""34""","""35""","""08""","""12""","""19""","""27""","""37""","""39""",false,0,"""09_06_08_10_01""",-6
"""02/01/2025""","""1751""","""10 34 24 16 35 01""","""11 03 18 29 15 28""","""""","""""","""011016243435""","[1, 10, … 35]",1,10,16,24,34,35,"""01""","""10""","""16""","""24""","""34""","""35""","""01""","""10""","""16""","""24""","""34""","""35""",false,0,"""09_06_08_10_01""",-6
"""19/12/2024""","""1745""","""10 13 25 35 20 30""","""25 11 10 23 31 17""","""""","""""","""101320253035""","[10, 13, … 35]",10,13,20,25,30,35,"""10""","""13""","""20""","""25""","""30""","""35""","""04""","""13""","""15""","""16""","""20""","""29""",true,8,"""03_07_05_05_05""",-10
"""19/12/2024""","""1745""","""10 13 25 35 20 30""","""25 11 10 23 31 17""","""""","""""","""101320253035""","[10, 13, … 35]",10,13,20,25,30,35,"""10""","""13""","""20""","""25""","""30""","""35""","""10""","""13""","""20""","""25""","""30""","""35""",true,8,"""03_07_05_05_05""",-10


df_jumps_map FOR str_b6 = 35


jumps_map,count
str,u32
"""02_10_09_04_06""",2
"""17_01_04_01_02""",2
"""03_06_02_13_03""",2
"""08_01_07_03_08""",2
"""02_03_02_04_05""",2
…,…
"""03_08_02_08_03""",1
"""02_07_02_05_07""",1
"""06_03_05_16_04""",1


df_unijump FOR str_b6 = 35


unijump,count
i64,u32
14,12
-21,9
6,9
21,9
29,8
…,…
-76,1
-66,1
-4,1


In [13]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [14]:
parquet_file = './parquet_kbl_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []

  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)


  with open('./gn_cahe_v3.json', 'w') as file: 
    json.dump(lst, file)

  
  # dfRows = pl.DataFrame(
  #   lst,
  #   schema=[
  #     'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
  #     'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
  #     'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
  #     'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
  #     'is_rare_combination', 'reason',
  #     "jumps_map", "unijump"
  #   ],
  #   schema_overrides={
  #     'combo': pl.Array(pl.UInt8, 6),
  #     'b1': pl.UInt8,
  #     'b2': pl.UInt8,
  #     'b3': pl.UInt8,
  #     'b4': pl.UInt8,
  #     'b5': pl.UInt8,
  #     'b6': pl.UInt8,
  #     'reason': pl.UInt8
  #   },
  #   orient='row'
  # )

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 6),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

In [15]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  pl.col("str_b5").is_in(possible_b5),
  pl.col("str_b6").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2),
  ~pl.col("str_b3").is_in(last_post_b3),
  ~pl.col("str_b4").is_in(last_post_b4),
  ~pl.col("str_b5").is_in(last_post_b5),
  ~pl.col("str_b6").is_in(last_post_blast),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_b5 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""010411232536""","[1, 4, … 36]",1,4,11,23,25,36,"""01""","""04""","""11""","""23""","""25""","""36""",false,0,"""03_07_12_02_11""",50
"""010411232537""","[1, 4, … 37]",1,4,11,23,25,37,"""01""","""04""","""11""","""23""","""25""","""37""",false,0,"""03_07_12_02_12""",51
"""010411232538""","[1, 4, … 38]",1,4,11,23,25,38,"""01""","""04""","""11""","""23""","""25""","""38""",false,0,"""03_07_12_02_13""",52
"""010411232539""","[1, 4, … 39]",1,4,11,23,25,39,"""01""","""04""","""11""","""23""","""25""","""39""",false,0,"""03_07_12_02_14""",53
"""010411232540""","[1, 4, … 40]",1,4,11,23,25,40,"""01""","""04""","""11""","""23""","""25""","""40""",false,0,"""03_07_12_02_15""",54
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""071722303538""","[7, 17, … 38]",7,17,22,30,35,38,"""07""","""17""","""22""","""30""","""35""","""38""",false,0,"""10_05_08_05_03""",21
"""071722303539""","[7, 17, … 39]",7,17,22,30,35,39,"""07""","""17""","""22""","""30""","""35""","""39""",false,0,"""10_05_08_05_04""",22
"""071722303738""","[7, 17, … 38]",7,17,22,30,37,38,"""07""","""17""","""22""","""30""","""37""","""38""",false,0,"""10_05_08_07_01""",11


In [16]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""070811253138""","[7, 8, … 38]",7,8,11,25,31,38,"""07""","""08""","""11""","""25""","""31""","""38""",false,0,"""01_03_14_06_07""",56
